In [1]:
import os 
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import re

from collections import Counter

In [2]:
%matplotlib inline

In [3]:
# Some of the columns we will look at can be quite wide, but it's good to get an idea of what they contain
print(pd.get_option('max_colwidth'))
pd.set_option('max_colwidth',500)

50


## File/dir locations


In [4]:
DATA_DIR = os.getenv("DATA_DIR")
filename = "preprocessed_taxon_pageseq_20190114_20190116.csv.gz"
# df_file = os.path.join(DATA_DIR, "processed_journey", filename)
# df_reduced_file = os.path.join(DATA_DIR, "processed_journey", "reduced_"+filename)
# df_rel_file = os.path.join(DATA_DIR, "processed_journey", "rel_"+filename)
df_doo_file = os.path.join(
    DATA_DIR, "processed_journey",
    "doo_prelim_meta_standard_with_pageseq_from_29-10_to_04-11-2018.csv.gz")

Load up a data file that isn't too large - we just want to check that the putative metrics for analysis can be derived from the data. This data was produced by an early version of the pipeline and is missing some descriptive variables, such as taxons etc. However, it contains the sequences of pages and behaviours (or events) of users on those pages, including interaction with the sidebar and the related links contained therein.

In [5]:
df = pd.read_csv(df_doo_file, compression="gzip")

In [6]:
df.head(2)

,Sequence,PageSequence,Occurrences,Page_Seq_Occurrences,DeviceCategories,Dates,Page_Event_List,Page_List,PageSequence_internal,Event_List,num_event_cats,Event_cats_agg,Event_cat_act_agg,Page_List_NL,Page_Seq_NL,Occurrences_NL
0,/guidance/rates-and-thresholds-for-employers-2016-to-2017<<PAGE<:<NULL<:<NULL,/guidance/rates-and-thresholds-for-employers-2016-to-2017,87,306.0,"[('desktop', 64), ('mobile', 11), ('tablet', 12)]","[('20181031', 33), ('20181029', 27), ('20181102', 17), ('20181104', 10)]","[('/guidance/rates-and-thresholds-for-employers-2016-to-2017', 'PAGE<:<NULL<:<NULL')]",['/guidance/rates-and-thresholds-for-employers-2016-to-2017'],/guidance/rates-and-thresholds-for-employers-2016-to-2017,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/guidance/rates-and-thresholds-for-employers-2016-to-2017'],/guidance/rates-and-thresholds-for-employers-2016-to-2017,6198.0
1,/sure-start-maternity-grant/how-to-claim<<PAGE<:<NULL<:<NULL>>/government/publications/sure-start-maternity-grant-claim-form<<PAGE<:<NULL<:<NULL>>/government/publications/sure-start-maternity-grant-claim-form<<EVENT<:<External Link Clicked<:<https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf>>/government/publications/sure-start-maternity-grant-claim-form<<EVENT<:<Download Link Clicked<:<https://assets.pu...,/sure-start-maternity-grant/how-to-claim>>/government/publications/sure-start-maternity-grant-claim-form,104,385.0,"[('mobile', 50), ('desktop', 50), ('tablet', 4)]","[('20181031', 34), ('20181029', 31), ('20181102', 24), ('20181104', 15)]","[('/sure-start-maternity-grant/how-to-claim', 'PAGE<:<NULL<:<NULL'), ('/government/publications/sure-start-maternity-grant-claim-form', 'PAGE<:<NULL<:<NULL'), ('/government/publications/sure-start-maternity-grant-claim-form', 'EVENT<:<External Link Clicked<:<https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf'), ('/government/publications/sure-start-maternity-grant-claim-form', 'EVENT<:<Download Link Clic...","['/sure-start-maternity-grant/how-to-claim', '/government/publications/sure-start-maternity-grant-claim-form']",/sure-start-maternity-grant/how-to-claim>>/government/publications/sure-start-maternity-grant-claim-form,"[('PAGE_NULL', 'PAGE_NULL'), ('PAGE_NULL', 'PAGE_NULL'), ('External Link Clicked', 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf'), ('Download Link Clicked', 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf')]",3,"[('PAGE_NULL', 2), ('External Link Clicked', 1), ('Download Link Clicked', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 2), (('External Link Clicked', 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf'), 1), (('Download Link Clicked', 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf'), 1)]","['/sure-start-maternity-grant/how-to-claim', '/government/publications/sure-start-maternity-grant-claim-form']",/sure-start-maternity-grant/how-to-claim>>/government/publications/sure-start-maternity-grant-claim-form,7010.0


In [7]:
df.shape

(2035522, 16)

## Remove tablet occurrences

Tablet data won't be in our future datasets so let's remove it here. We won't be using it in future because depending on screen width/orientation it can display the page in a "mobile" or "desktop" format, where related links are in different places, this will make analysis of the data more complicated.

In [8]:
def device_count(x, device):
    return sum([value for item, value in x if item == device])

df["TabletCount"] = df['DeviceCategories'].apply(
    ast.literal_eval).map(lambda x: device_count(x, "tablet"))
df["Occurrences"] = df["Occurrences"] - df["TabletCount"]
df = df[df["Occurrences"] != 0]
df.shape

(1893413, 17)

## Planning

Inspecting our putative metrics for detecting a change in user navigation experience gives us three ratios. Thus we need to check that we can get the numerator and denominator from the data for each of these metrics.


In [9]:
df['Event_cat_act_agg']= df['Event_cat_act_agg'].apply(ast.literal_eval)

In [ ]:
events = []
for items in df['Event_cat_act_agg']:
    for event in items:
        events.append(event[0])

In [ ]:
events[:4]

In [ ]:
# look at counter of events
# Counter(events)

Related link events eventCategory:
- relatedLinkClicked (eventAction e.g. 1.3 Related content)

There is more than one level to this Event, we are specifically interested in "Related content" (as this is the sidebar of the page, the related links we are interested in).

### Navigation events
Potential nav events eventCategory:
- breadcrumbClicked
- homeLinkClicked -  First full day of data 08 Nov 2018
- searchResults - deprecated according to [GOV.UK wiki](https://gov-uk.atlassian.net/wiki/spaces/GOVUK/pages/23855552/Analytics+on+GOV.UK#AnalyticsonGOV.UK-Events), but still firing on the site - **doesn't fire if there are no search results, so let's use page = search instead**  i.e. '/search?q=' in page
- relatedLinkClicked (eventAction e.g. 1.1 Explore the topic) - Explore the topic links at the bottom of a content page

**What events correspond to the links in grey right at the bottom of the page?**

more events to look into:
- ~~firstLevelBrowseLinkClicked~~  relates to browse page
- ~~secondLevelBrowseLinkClicked~~ relates to browse page

- ~~navDocumentCollectionLinkClicked~~ click on a link that leads you to a collection of documents, e.g from https://www.gov.uk/government/collections/data-protection-act-2018 
- ~~navFinderLinkClicked~~ click a link from a finder's results, eg. from https://www.gov.uk/raib-reports
- ~~navAccordionLinkClicked~~ expand a section on a page e.g. https://www.gov.uk/world/germany#/world/emergency-help-for-british-nationals-germany
- ~~navLeafLinkClicked~~ click on a section from a taxon page like https://www.gov.uk/world/brexit-ireland
- ~~navServicesInformationLinkClicked~~ click a link from a services_and_information page like https://www.gov.uk/government/organisations/hm-revenue-customs/services-information
- ~~navSubtopicContentItemLinkClicked~~	- click on a content item from a topic page, e.g. 'Self-employed National Insurance rates' from https://www.gov.uk/topic/personal-tax/national-insurance
- ~~navSubtopicLinkClicked~~ a link to a subtopic from a topic page is clicked, e.g from https://www.gov.uk/topic/working-sea
- navTopicLinkClicked not in this sample
- ~~searchBoxFilter~~ use the search box on a page like https://www.gov.uk/world
- ~~navGridContentClicked~~ exploring a subtopic from a topic page, e.g. the grid at the bottom of https://www.gov.uk/environment/climate-change-energy


**step by step - do not include?**
- ~~stepNavHeaderClicked~~ - if a content page is part of a step by step, this is clicking the link to the step by step at the top of the page - does this indicate they are lost or they are where they want to be?
- ~~stepNavLinkClicked~~ - a link to click through to another part of the step by step is clicked
- ~~stepNavPartOfClicked~~ - sidebar part of step by step link clicked

In [ ]:
# look at search pages
def contains_search(x):
    return '/search?q=' in x 

In [ ]:
# df[df['Sequence'].map(contains_search)]

In [ ]:
# use this fn to look at rows in the DF that contain a specific event category, or 
# anything else you want to find in 'Sequence', then you can look at the pages to 
# work out how the event can occur
def look_at_event(event_cat):
    return df[df['Sequence'].map(lambda x: event_cat in x)].head(5)

In [ ]:
# df[df['Sequence'].map(lambda x: '>>/<<' in x)]

### Page types

We want to know what page types different pages are because we care about navigation elements clicked **from a content page** and the total number of journeys that contain **only content pages and related links**.

Initially we wanted to use [User journey supertype](https://docs.publishing.service.gov.uk/document-types/user_journey_document_supertype.html) from [custom dimensions](https://gov-uk.atlassian.net/wiki/spaces/GOVUK/pages/23855552/Analytics+on+GOV.UK#AnalyticsonGOV.UK-customDimensionsCustomdimensions), where each page is either classified as 'finding' or 'thing'. Unfortunately this dimension is poorly populated in BigQuery due to someone disabling it, and occasionally the same page path and page title can be reported as both a 'finding' and a 'thing' at different times (e.g. the GOV.UK homepage).

The next option is using 'Format' (document type) from [custom dimensions](https://gov-uk.atlassian.net/wiki/spaces/GOVUK/pages/23855552/Analytics+on+GOV.UK#AnalyticsonGOV.UK-customDimensionsCustomdimensions), which is a lot better populated but a lot more granular, you can find a sample of a day's data with pageviews from GA [here](https://docs.google.com/spreadsheets/d/1-jen8DbRgvmvF9aYapmwsFAe0ncHeq5y-ks75iyCvVU/edit#gid=177174931).

*NB - a data dump from the content API is probably the most reliable source for a mapping of page paths to document types and then to finding/thing, however there's a bit of outlay in understanding how that data can be accessed and how it is structured so a BigQuery query is most appropriate for a first stab at this.*

**TODO** one query to get pagePath, format (or map this to finding/thing), content ID, where content_id != '00000000-0000-0000-0000-000000000000'

In [ ]:
ProjectID = 'govuk-bigquery-analytics'
KEY_DIR = os.getenv("BQ_KEY_DIR")
key_file_path = os.path.join(KEY_DIR, os.listdir(KEY_DIR)[0])

In [ ]:
# OLD_QUERY = """
#     SELECT DISTINCT
#     content_id,
#     pagePath,
#     IF(document_type IN ('document_collection',
#       'finder',
#       'homepage',
#       'license_finder',
#       'mainstream_browse_page',
#       'organisation',
#       'search',
#       'service_manual_homepage',
#       'service_manual_topic',
#       'services_and_information',
#       'taxon',
#       'topic',
#       'topical_event'),'finding','thing') as finding_or_thing
#     FROM
#       (SELECT 
#           (
#         SELECT
#           value
#         FROM
#           hits.customDimensions
#         WHERE
#           index=4) AS content_id,
#           hits.page.pagePath, 
#         (
#         SELECT
#           value
#         FROM
#           hits.customDimensions
#         WHERE
#           index=2) AS document_type
#       FROM
#         `govuk-bigquery-analytics.87773428.ga_sessions_*` AS sessions
#       CROSS JOIN
#         UNNEST(sessions.hits) AS hits
#          WHERE _TABLE_SUFFIX BETWEEN '20190121'
#      AND '20190127'
#       )
#     WHERE
#       content_id != '00000000-0000-0000-0000-000000000000'
#       AND content_id != '[object Object]'
#       AND content_id IS NOT NULL
# """

In [ ]:
# QUERY = """
#     SELECT 
#     pagePath,
#     MAX(IF(document_type IN ('document_collection',
#       'finder',
#       'homepage',
#       'license_finder',
#       'mainstream_browse_page',
#       'organisation',
#       'search',
#       'service_manual_homepage',
#       'service_manual_topic',
#       'services_and_information',
#       'taxon',
#       'topic',
#       'topical_event'),1,0)) AS is_finding
#     FROM
#       (SELECT 
#           (
#         SELECT
#           value
#         FROM
#           hits.customDimensions
#         WHERE
#           index=4) AS content_id,
#           hits.page.pagePath, 
#         (
#         SELECT
#           value
#         FROM
#           hits.customDimensions
#         WHERE
#           index=2) AS document_type
#       FROM
#         `govuk-bigquery-analytics.87773428.ga_sessions_*` AS sessions
#       CROSS JOIN
#         UNNEST(sessions.hits) AS hits
#          WHERE _TABLE_SUFFIX BETWEEN '20190121'
#      AND '20190127'
#       )
#     WHERE
#       content_id != '00000000-0000-0000-0000-000000000000'
#       AND content_id != '[object Object]'
#       AND content_id IS NOT NULL
#     GROUP BY 1
# """

In [ ]:
# df_finding_thing = pd.io.gbq.read_gbq(QUERY,
#                            project_id=ProjectID,
#                            reauth=False,
#                            # verbose=True,
#                            private_key=key_file_path,
#                            dialect='standard')

In [ ]:
# df_finding_thing.to_csv('../../data/raw_bq_extract/document_types.csv.gz',
#              sep="\t", index=False, compression='gzip')

In [9]:
df_finding_thing = pd.read_csv(
    '../../data/raw_bq_extract/document_types.csv.gz',
             sep="\t", compression='gzip')

## journey_click_rate
There is no difference in the proportion of journeys using at least one related link (journey_click_rate) between page variant A and page variant B.



\begin{equation*}
\frac{\text{total number of journeys including at least one click on a related link}}{\text{total number of journeys}}
\end{equation*}

### total number of journeys including at least one click on a related link
The numerator.

We need to check within the Sequence column, whether the corresponding user journey has an Event where a related link was clicked. There is more than one level to this Event, we are specifically interested in "Related content" (as this is the sidebar of the page, the related links we are interested in).

In [ ]:
#Compute whether a journey includes at least one related link click
def is_related(x):
    return all(cond in x for cond in ["relatedLinkClicked","Related content"])

Please note, `is_related` does not make sure that `relatedLinkClicked` and `Related content` exist in the same event in `Sequence`, example below

In [ ]:
is_related(
    '/garden-bonfires-rules<<PAGE<:<relatedLinkClicked<:<NULL>>/garden-bonfires-rules<<EVENT<:<NULL<:<1.4 Related content'
    )

In [ ]:
# map across the Sequence variable, which includes pages and Events
# we want to pass all the list elements to a function one-by-one and then collect the output.
df["Has_Related"] = df["Sequence"].map(is_related)

In [ ]:
# this gives a logical column
df["Has_Related"].head()

In [ ]:
# We can filter for True and sum
df[df["Has_Related"]].Occurrences.sum()

In [ ]:
# sense check by inspecting filtered df
df[df["Has_Related"]].head(3)

### total number of journeys
The denominator.

In [ ]:
df.Occurrences.sum()

### final metric

Given this sample, we see:

In [ ]:
df[df["Has_Related"]].Occurrences.sum() / df.Occurrences.sum()

The above metric considers at least one related link clicked, but on some journeys more than one related link might get clicked. We can use the following columns to help us.

In [ ]:
df['Event_cats_agg'].head(3)

In [ ]:
df['Event_cat_act_agg'].head(3)

## ratio of clicks on navigation elements vs. clicks on related links

There is no statistically significant difference in the ratio of clicks on navigation elements vs. clicks on related links between page variant A and page variant B

\begin{equation*}
\frac{\text{total number of navigation element click events from content pages}}{\text{total number of related link click events}}
\end{equation*}

### total number of related link click events

we need to check `Related content` is in the event, because the `relatedLinkClicked` is also used for "explore the topic" links at the bottom of the page, with the event action containing `Explore the topic`, e.g. `(('relatedLinkClicked', '2.1 Explore the topic'), 1)`

In [ ]:
# example of Event_cat_act_agg
df['Event_cat_act_agg'][2608]

In [ ]:
# If the event category is 'relatedLinkClicked' and the event action contains 'Related content', 
# return the count of that event
def get_number_of_events_rl(event):
    if event[0][0] == 'relatedLinkClicked' and 'Related content' in event[0][1]:
        return event[1]
    return 0

In [ ]:
def sum_related_click_events(event_list):
    return sum([get_number_of_events_rl(event) for event in event_list])

In [ ]:
# get the number of related links clicks per Sequence
df['Related Links Clicks per seq'] = df['Event_cat_act_agg'].map(sum_related_click_events)

# get the total number of related links clicks for that row (clicks per sequence multiplied by occurrences)
df['Related Links Clicks row total'] = df['Related Links Clicks per seq'] * df['Occurrences']

In [ ]:
df[df["Has_Related"]].head(3)

In [ ]:
df['Related Links Clicks row total'].sum()

### total number of navigation element click events from content pages

Page_Event_List:	from Sequence -> list of tuples of (page url, event). Where it's a page hit, event==PAGE_NULL


eventCategory == 'breadcrumbClicked'

eventCategory == 'homeLinkClicked'

eventCategory == 'relatedLinkClicked',  'Explore the topic' in Event Action

'/search?q=' in page

In [10]:
thing_page_paths = df_finding_thing[
    df_finding_thing['is_finding']==0]['pagePath'].tolist()

In [60]:
finding_page_paths = df_finding_thing[
    df_finding_thing['is_finding']==1]['pagePath'].tolist()

#### Nav events

In [ ]:
df['Page_Event_List'] = df['Page_Event_List'].apply(ast.literal_eval)

In [ ]:
example_Page_Event_List = df['Page_Event_List'][9315]

In [ ]:
def is_nav_event(event):
    return any(
        ['breadcrumbClicked' in event, 'homeLinkClicked' in event,
         all(cond in event for cond in [
             'relatedLinkClicked','Explore the topic'])])

In [ ]:
print(is_nav_event('EVENT<:<breadcrumbClicked<:<2'))
print(is_nav_event('PAGE<:<NULL<:<NULL'))
print(is_nav_event('EVENT<:<relatedLinkClicked<:<2.3 Explore the topic'))
print(is_nav_event('EVENT<:<relatedLinkClicked<:<2.3 Related content'))
print(is_nav_event('EVENT<:<homeLinkClicked<:<homeHeader'))

In [ ]:
content_page_nav_events = 0
for pair in example_Page_Event_List:
    if is_nav_event(pair[1]):
        if pair[0] in thing_page_paths:
            content_page_nav_events += 1

In [ ]:
print(example_Page_Event_List)
print(content_page_nav_events)

In [ ]:
def count_nav_events(page_event_list):
    content_page_nav_events = 0
    for pair in page_event_list:
        if is_nav_event(pair[1]):
            if pair[0] in thing_page_paths:
                content_page_nav_events += 1
    return content_page_nav_events

In [ ]:
df['Content_page_nav_events'] = df['Page_Event_List'].map(count_nav_events)

In [ ]:
# df[df['Content_page_nav_events'] >2]

**There are some weird looped sequences in the data**

In [ ]:
df['Page_Event_List'][139307]

#### Search pages

thing_page_paths is a list of page paths of document type "thing"

'Page_List' is a column with a list of page paths (in order) that were visited in each journey

all search pages paths contain '/search?q=' at the beginning

In [11]:
df['Page_List'] = df['Page_List'].apply(ast.literal_eval)

In [28]:
def count_search_from_content(page_list):
    search_from_content = 0
    for i, page in enumerate(page_list):
        if i > 0:
            if '/search?q=' in page:
                if page_list[i-1] in thing_page_paths:
                    search_from_content += 1
    return search_from_content

def count_search_from_content_2(page_list):
    search_from_content = 0
    for i, page in enumerate(page_list[1:]):
        if '/search?q=' in page:
            if page_list[i] in thing_page_paths:
                search_from_content += 1
    return search_from_content

def count_search_from_content_3(page_list):
    search_from_content = 0
    if '/search?q=' in ''.join(page_list):
        for i, page in enumerate(page_list[1:]):
            if '/search?q=' in page:
                if page_list[i] in thing_page_paths:
                    search_from_content += 1
    return search_from_content

def count_search_from_content_4(page_list):
    search_from_content = 0
    if len(page_list) > 1:
        for i, page in enumerate(page_list[1:]):
            if '/search?q=' in page:
                if page_list[i] in thing_page_paths:
                    search_from_content += 1
    return search_from_content

def count_search_from_content_5(page_list):
    search_from_content = 0
    if '/search?q=' in ''.join(page_list):
        for i, page in enumerate(page_list[1:]):
            if all(['/search?q=' in page, page_list[i] in thing_page_paths]):
                    search_from_content += 1
    return search_from_content

In [42]:
df['pageviews'] = df['Page_List'].map(len)

In [43]:
_10000_sample_df = df.sample(n=10000, weights=df.Occurrences)

In [21]:
# _10000_sample_df['Search_from_content_events_2'] = _10000_sample_df['Page_List'].map(count_search_from_content_2)

In [ ]:
# _10000_sample_df[_10000_sample_df['Search_from_content_events_2']>0]

In [52]:
%timeit _10000_sample_df['Page_List'].map(count_search_from_content)

2.53 s ± 144 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
%timeit _10000_sample_df['Page_List'].map(count_search_from_content_2)

2.6 s ± 65.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%timeit _10000_sample_df['Page_List'].map(count_search_from_content_3)

2.55 s ± 149 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
%timeit _10000_sample_df['Page_List'].map(count_search_from_content_4)

2.62 s ± 50.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
%timeit _10000_sample_df['Page_List'].map(count_search_from_content_5)

6.11 s ± 338 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
%timeit _10000_sample_df[_10000_sample_df['pageviews'] > 1]['Page_List'].map(count_search_from_content)

2.65 s ± 92.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
%timeit _10000_sample_df[_10000_sample_df['pageviews'] > 1]['Page_List'].map(count_search_from_content_2)

2.61 s ± 60.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%%timeit 
_10000_sample_df[_10000_sample_df['pageviews'] > 1]['Page_List'].map(count_search_from_content_3)

2.65 s ± 53.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### Trying out regexes

The above methods took a while, so I tried to make a super-regex of all combinations of content page followed by search page, but this ended up taking even longer

In [24]:
eg = '/content1>>/search?q=abc>>/dsfjk>>/content2>>/bleh?q=dfe'

In [25]:
re.findall('\/content1>>\/search\?q=|\/content2>>\/search\?q=', eg)

['/content1>>/search?q=']

In [26]:
%timeit len(re.findall('\/content1>>\/search\?q=|\/content2>>\/search\?q=', eg))

1.33 µs ± 23.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [29]:
%timeit count_search_from_content_3('\/content1>>\/search\?q=|\/content2>>\/search\?q=')

1.21 µs ± 12.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [30]:
%timeit len(re.findall('(\/content1>>|\/content2>>)\/search\?q=', eg))

1.31 µs ± 44.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [32]:
%timeit len(re.findall('(\/content1|\/content2)>>\/search\?q=', eg))

1.31 µs ± 23.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
thing_page_paths_escaped = df_finding_thing[
    df_finding_thing['is_finding']==0]['pagePath'].map(re.escape).tolist()

In [57]:
thing_page_paths_escaped[:5]

['\\/jobsearch',
 '\\/pay\\-leave\\-for\\-parents\\/y',
 '\\/student\\-finance\\-calculator\\/y\\/2018\\-2019\\/uk\\-full\\-time',
 '\\/government\\/publications\\/application\\-for\\-confirmation\\-of\\-british\\-nationality\\-status\\-form\\-ns',
 '\\/pip\\/eligibility']

In [33]:
re.findall(
    '(\\' +'>>|\\'.join(['\\/content1', '\\/content2'])+'>>)\\/search\\?q='
    , eg)


[]

In [14]:
regex_thing_then_search = '(\\' +'>>|'.join(thing_page_paths_escaped)+'>>)\\/search\\?q='

In [15]:
compiled_regex = re.compile(regex_thing_then_search)

In [20]:
def count_search_from_content_regex(page_sequence):
    return len(compiled_regex.findall(page_sequence))

In [22]:
# this took 48 mins
# _10000_sample_df['PageSequence'].map(count_search_from_content_regex)

## count of related links clicked by journey

Using work from "total number of related link click events" section above, below is some information about the distribution of total related link clicks in a journey. This is to help decide how we might model this metric.

In [ ]:
group_by_rl_clicks_df = df.groupby(by='Related Links Clicks per seq').sum().reset_index()

In [ ]:
group_by_rl_clicks_df

In [ ]:
group_by_rl_clicks_df_bar_zero = group_by_rl_clicks_df[
    group_by_rl_clicks_df['Related Links Clicks per seq'] != 0]

In [ ]:
plt.bar(
    group_by_rl_clicks_df['Related Links Clicks per seq'],
    group_by_rl_clicks_df['Occurrences'],
    align='edge')
plt.title('Related link clicks per journey')
plt.ylabel('number of journeys')
plt.xlabel('number of related link clicks in a journey')

In [ ]:
plt.bar(
    group_by_rl_clicks_df['Related Links Clicks per seq'],
    group_by_rl_clicks_df['Occurrences'],
    log=True
    ,align='edge'
)
plt.title('Related link clicks per journey - logscale')
plt.ylabel('log(number of journeys)')
plt.xlabel('number of related link clicks in a journey')

In [ ]:
plt.bar(
    group_by_rl_clicks_df_bar_zero['Related Links Clicks per seq'],
    group_by_rl_clicks_df_bar_zero['Occurrences'],
    align='edge')
plt.title('Related link clicks per journey - excluding zero clicks')
plt.ylabel('number of journeys')
plt.xlabel('number of related link clicks in a journey')

In [ ]:
plt.bar(
    group_by_rl_clicks_df_bar_zero['Related Links Clicks per seq'],
    group_by_rl_clicks_df_bar_zero['Occurrences'],
    log=True
    ,align='edge'
)
plt.title('Related link clicks per journey - excluding zero clicks, logscale')
plt.ylabel('log(number of journeys)')
plt.xlabel('number of related link clicks in a journey')

## proportion of journeys with a page sequence including content and related links only

There is no statistically significant difference in the proportion of journeys with a page sequence including content and related links only (including loops) between page variant A and page variant B

\begin{equation*}
\frac{\text{total number of journeys that only contain content pages and related links (i.e. no nav pages)}}{\text{total number of journeys}}
\end{equation*}

### total number of journeys that only contain content pages and related links (i.e. no nav pages)
This will be easier to calculate as total number of journeys that do not contain clicks on navigation elements (home, breadcrumbs, search, explore this topic)

Could we look at journeys without any 'finding' pages? (finding includes search pages)

**In the future perhaps we could add document type to our BigQuery queries and pipeline - we could have a document_type_list - an ordered list of document types of each page, and perhaps one that is just finding/thing, or finding(excl search)/thing/search**

In [61]:
def any_finding_pages(page_list):
    return any([page_path in finding_page_paths for page_path in page_list])

In [ ]:
df['Any_finding_Page'] = df['Page_List'].map(any_finding_pages)

In [ ]:
df.head()

In [83]:
def is_nav_event_cat_act_agg(event):
    return any([
        [event[0][0] in ['breadcrumbClicked', 'homeLinkClicked'],
                event[0][0] == 'relatedLinkClicked' and 'Explore the topic' in event[0][1]
               ]]
              )

In [84]:
def any_nav_events(event_cat_act_agg):
    return any(
        [is_nav_event_cat_act_agg(event) for event in event_cat_act_agg])

In [85]:
df['Any_nav_event'] =  df['Event_cat_act_agg'].map(any_nav_events)

In [86]:
df[df['Any_nav_event'] == True]

,Sequence,PageSequence,Occurrences,Page_Seq_Occurrences,DeviceCategories,Dates,Page_Event_List,Page_List,PageSequence_internal,Event_List,num_event_cats,Event_cats_agg,Event_cat_act_agg,Page_List_NL,Page_Seq_NL,Occurrences_NL,TabletCount,pageviews,Any_nav_event
0,/guidance/rates-and-thresholds-for-employers-2016-to-2017<<PAGE<:<NULL<:<NULL,/guidance/rates-and-thresholds-for-employers-2016-to-2017,75,306.0,"[('desktop', 64), ('mobile', 11), ('tablet', 12)]","[('20181031', 33), ('20181029', 27), ('20181102', 17), ('20181104', 10)]","[('/guidance/rates-and-thresholds-for-employers-2016-to-2017', 'PAGE<:<NULL<:<NULL')]",[/guidance/rates-and-thresholds-for-employers-2016-to-2017],/guidance/rates-and-thresholds-for-employers-2016-to-2017,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/guidance/rates-and-thresholds-for-employers-2016-to-2017'],/guidance/rates-and-thresholds-for-employers-2016-to-2017,6.198000e+03,12,1,True
1,/sure-start-maternity-grant/how-to-claim<<PAGE<:<NULL<:<NULL>>/government/publications/sure-start-maternity-grant-claim-form<<PAGE<:<NULL<:<NULL>>/government/publications/sure-start-maternity-grant-claim-form<<EVENT<:<External Link Clicked<:<https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf>>/government/publications/sure-start-maternity-grant-claim-form<<EVENT<:<Download Link Clicked<:<https://assets.pu...,/sure-start-maternity-grant/how-to-claim>>/government/publications/sure-start-maternity-grant-claim-form,100,385.0,"[('mobile', 50), ('desktop', 50), ('tablet', 4)]","[('20181031', 34), ('20181029', 31), ('20181102', 24), ('20181104', 15)]","[('/sure-start-maternity-grant/how-to-claim', 'PAGE<:<NULL<:<NULL'), ('/government/publications/sure-start-maternity-grant-claim-form', 'PAGE<:<NULL<:<NULL'), ('/government/publications/sure-start-maternity-grant-claim-form', 'EVENT<:<External Link Clicked<:<https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf'), ('/government/publications/sure-start-maternity-grant-claim-form', 'EVENT<:<Download Link Clic...","[/sure-start-maternity-grant/how-to-claim, /government/publications/sure-start-maternity-grant-claim-form]",/sure-start-maternity-grant/how-to-claim>>/government/publications/sure-start-maternity-grant-claim-form,"[('PAGE_NULL', 'PAGE_NULL'), ('PAGE_NULL', 'PAGE_NULL'), ('External Link Clicked', 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf'), ('Download Link Clicked', 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf')]",3,"[('PAGE_NULL', 2), ('External Link Clicked', 1), ('Download Link Clicked', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 2), (('External Link Clicked', 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf'), 1), (('Download Link Clicked', 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/749722/sure-start-maternity-grant-sf100.pdf'), 1)]","['/sure-start-maternity-grant/how-to-claim', '/government/publications/sure-start-maternity-grant-claim-form']",/sure-start-maternity-grant/how-to-claim>>/government/publications/sure-start-maternity-grant-claim-form,7.010000e+03,4,2,True
2,/historic-vehicles/apply-for-vehicle-tax-exemption<<PAGE<:<NULL<:<NULL,/historic-vehicles/apply-for-vehicle-tax-exemption,100,363.0,"[('mobile', 63), ('desktop', 37), ('tablet', 15)]","[('20181031', 35), ('20181029', 25), ('20181102', 36), ('20181104', 19)]","[('/historic-vehicles/apply-for-vehicle-tax-exemption', 'PAGE<:<NULL<:<NULL')]",[/historic-vehicles/apply-for-vehicle-tax-exemption],/historic-vehicles/apply-for-vehicle-tax-exemption,"[('PAGE_NUL

## Other stuff

### total number of pageviews
Not sure we're using this in our metrics

Should these only include ones that HAD related links on them??

If we had col PageSeq_Length we could use that instead

In [ ]:
len(df['Page_List'][26010])

In [40]:
df['pageviews'] = df['Page_List'].map(len)

In [ ]:
df['absolute pageviews count'] = df['pageviews'] * df['Occurrences']

In [ ]:
df.head(2)

In [ ]:
# total pageviews
df['absolute pageviews count'].sum()